# Building a Tavily Data Agent


Adapted from <a href="https://colab.research.google.com/github/run-llama/llama-hub/blob/main/llama_hub/tools/notebooks/tavily.ipynb" target="_parent">this tutorial</a>, and the accompanying <a href="https://colab.research.google.com/github/run-llama/llama-hub/blob/main/llama_hub/tools/notebooks/tavily.ipynb" target="_parent">Colab link</a>.

This notebook guides you through using the LLM tools provided by the [Tavily API](https://app.tavily.com/) to enable LLMs to easily search and retrieve relevant content from the Internet.

To get started, you will need a [Tavily API key](https://app.tavily.com), and you might need an [OpenAI api key](https://platform.openai.com/account/api-keys) if you want to use models from openai.

We will import the relevant agents and tools and pass them our keys here:

In [ ]:
# !pip install llama-index llama-hub llama-index-agent-lats tavily-python llama-index-llms-ollama llama-index-tools-tavily_research

In [28]:
import getpass
import os


def _set_if_undefined(var: str) -> None:
    if os.environ.get(var):
        return
    os.environ[var] = getpass.getpass(var)

# _set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("TAVILY_API_KEY")

In [ ]:
# Set up OpenAI
# import openai
# from llama_index.agent import OpenAIAgent
# openai.api_key = "sk-api-key"

In [99]:
# !pip install llama-index-llms-ollama
# %pip install llama-index-llms-langchain

# use ollama
from llama_index.llms.ollama import Ollama

llm = Ollama(model="llama3:instruct", request_timeout=600.0)

# !pip install llama-index-llms-lmstudio
# from llama_index.llms.lmstudio import LMStudio
# from llama_index.core.base.llms.types import ChatMessage, MessageRole

# use LMStudio
# llm = LMStudio(
#     model_name="llama3:instruct",
#     base_url="http://localhost:1234/v1",
#     temperature=0.1,
#     request_timeout=600.0
# )

# use langchain ollama
# from llama_index.llms.langchain import LangChainLLM
# from langchain_community.chat_models import ChatOllama
# from langchain_experimental.llms.ollama_functions import OllamaFunctions

# # llm = ChatOllama(model="llama3:instruct", temperature=0.1)

# llm_with_tools = OllamaFunctions(model="llama3:instruct", format="json")

# llm = LangChainLLM(llm=llm_with_tools)

In [100]:
# Set up Tavily tool
from llama_index.tools.tavily_research import TavilyToolSpec
tavily_tool = TavilyToolSpec(
    api_key=os.environ["TAVILY_API_KEY"],
)

tavily_tool_list = tavily_tool.to_tool_list()
for tool in tavily_tool_list:
    print(tool.metadata.name)

search


## Testing the Tavily search tool

We've imported our OpenAI agent, set up the api key, and initialized our tool. Let's test out the tool before setting up our Agent.


In [30]:
tavily_tool.search("What happened in the latest Burning Man festival?", max_results=3)

[Document(id_='b84aedef-b878-49c9-9fff-e397ba6aa928', embedding=None, metadata={'url': 'https://www.today.com/news/what-is-burning-man-flood-death-rcna103231'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='"\nAn update on BurningMan.org posted the night of Sept. 3 offered those still on site a "Wet Playa Survival Guide" with tips, as well as an update saying that conditions were "improving" and that roads in Black Rock City were expected to open on Sept. 4 for "Exodus."\n"Thank goodness this community knows how to take care of each other," the Instagram page for Burning Man Information Radio wrote on a post predicting more rain.\nNeal Katyal, Chris Rock and others detail their ‘escape’ from Black Rock City\nComedian Chris Rock and Diplo, producer and DJ,\xa0were among this year\'s Burning Man attendees. Why are people stranded at Burning Man?\nEach year participants in the Burning Man event gather for nine days in an "annual experiment in temp

### Using the Search tool in an Agent

We can create an agent with access to the Tavily search tool start testing it out:

In [5]:
agent = OpenAIAgent.from_tools(
    tavily_tool_list,
)

In [7]:
print(
    agent.chat(
        "Write a deep analysis in markdown syntax about the latest burning man floods"
    )
)

# Analysis of the Latest Burning Man Floods

## Introduction
The Burning Man festival, held annually in the Black Rock Desert of Nevada, is known for its unique art installations, music performances, and community spirit. However, in recent years, the festival has faced challenges due to extreme weather conditions, including floods. This analysis aims to provide a deep understanding of the latest Burning Man floods, their causes, impacts, and the response of the festival organizers.

## Overview of the Latest Burning Man Floods
According to a [news article](https://www.today.com/news/what-is-burning-man-flood-death-rcna103231), the latest Burning Man festival experienced heavy rains that resulted in muddy conditions, making it difficult for festivalgoers to leave the Nevada desert. The article mentions that tens of thousands of festival attendees were slowly making their way home as the muddy conditions made it nearly impossible to leave over the weekend. The festival organizers provid

In [61]:
from llama_index.agent.lats import LATSAgentWorker

tools = [tavily_tool]

In [101]:
from llama_index.core.agent import AgentRunner
# LATSAgentWorker(
#     tools: Sequence[BaseTool]	| Tools to use for reasoning	required
#     llm: LLM | LLM to use   required
#     callback_manager: CallbackManager |	Callback manager    None
#     tool_retriever: Optional[ObjectRetriever[BaseTool]] | Tool retriever	None
#     verbose : bool | Whether to print out reasoning steps	False)

# class methods: from_tools(tools: Optional[Sequence[BaseTool]] = None, 
#                           tool_retriever: Optional[ObjectRetriever[BaseTool]] = None, 
#                           llm: Optional[LLM] = None, callback_manager: Optional[CallbackManager] = None, 
#                           verbose: bool = False, **kwargs: Any) -> CustomSimpleAgentWorker
# check for more methods: https://docs.llamaindex.ai/en/stable/api_reference/agent/#llama_index.core.agent.CustomSimpleAgentWorker

agent_worker = LATSAgentWorker.from_tools(
    tavily_tool.to_tool_list(),
    llm=llm,
    num_expansions=2,
    max_rollouts=2,  # using -1 for unlimited rollouts
    verbose=False,
)
# check https://llamahub.ai/l/agent/llama-index-agent-lats?from=agent
agent = AgentRunner(agent_worker)

# agent = agent.as_worker()

In [ ]:
# !pip install nest-asyncio
import nest_asyncio
nest_asyncio.apply()

res = agent.chat(question)
res

In [ ]:
task = agent.create_task(question)
step_output = agent.run_step(task.task_id)
while not step_output.is_last:
    try:
        step_output = agent.run_step(task.task_id)
    except Exception as e:
        print(e)

response = agent.finalize_response(task.task_id)

# Warnig this method is computational expensive, use with caution.